In [1]:
!nvidia-smi

Thu Jun 13 03:11:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.70                 Driver Version: 537.70       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   39C    P8               4W /  35W |      0MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
import datasets
import tempfile
import logging
import random
# import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
# import jsonlines

from utilities import *
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
# from transformers import AutoModelForCausalLM
# from llama import BasicModelRunner


logger = logging.getLogger(__name__)
global_config = None

C:\Users\LENOVO\.conda\envs\testing_finetune_llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#constants

filename = "dataset/lamini_docs.jsonl"
model_name = "EleutherAI/pythia-70m"
model_save_path = f"saved_base_model_with_model_name_{model_name}.pth"
max_steps = 3
trained_model_name = f"lamini_docs_{max_steps}_steps"
output_dir = trained_model_name

# dataset_name = "lamini_docs.jsonl"
# dataset_path = f"/content/{dataset_name}"
# use_hf = False
dataset_path = "lamini/lamini_docs"
use_hf = False #True

training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": filename #dataset_path
    },
    "verbose": True
}


In [ ]:
# Download dataset from hugging face hub

dataset_path = "lamini/lamini_docs"
dataset = datasets.load_dataset(dataset_path)
print(dataset)

In [6]:
# load the dataset from local and split the dataset into training and testing set

finetuning_dataset_loaded = datasets.load_dataset("json", data_files=filename, split="train")
split_dataset = finetuning_dataset_loaded.train_test_split(test_size=0.1, shuffle=True, seed=123)

train_dataset, test_dataset = split_dataset['train'], split_dataset['test']

print(train_dataset)
print(test_dataset)

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1260
})
Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 140
})


In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

C:\Users\LENOVO\.conda\envs\testing_finetune_llm\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
device_count = torch.cuda.device_count()
if device_count > 0:
    # logger.debug("Select GPU device")
    print("Select GPU device")
    device = torch.device("cuda")
else:
    # logger.debug("Select CPU device")
    device = torch.device("cpu")

Select GPU device


In [9]:
# Load model from trasnformer (hugging face hub)

base_model = AutoModelForCausalLM.from_pretrained(model_name)

In [9]:
# load the pretrained model and tokenizer from local

save_directory = f"pretrain_model/{model_name}"
model = AutoModelForCausalLM.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)

In [10]:
base_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [11]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [13]:
# if download dataset from hugging face 

# test_text = dataset['test'][0]['question']
# print("Question input (test):", test_text)
# print(f"Correct answer from Lamini docs: {dataset['test'][0]['answer']}")
# print("Model's answer: ")
# print(inference(test_text, base_model1, tokenizer))

# if data is loaded from local
test_text = test_dataset[0]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {test_dataset[0]['answer']}")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

Question input (test): Is it possible to fine-tune Lamini on a specific dataset for text generation in legal documents?
Correct answer from Lamini docs: Lamini’s LLM Engine can help you fine-tune any model on huggingface or any OpenAI model.
Model's answer: 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




A:

I think you should use the following code:
public class TextFile : File
{
    public string FileName { get; set; }
    public string TextFileName { get; set; }
    public string TextFileName { get; set; }
    public string TextFileName { get; set; }
    public


In [14]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=10,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  # max_steps=1,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

In [16]:
model_flops = (
  base_model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, training_config["model"]["max_length"])
      )
    }
  )
  * training_args.gradient_accumulation_steps
)

print(base_model)
print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [17]:
trainer = Trainer(
    model=base_model,
    model_flops=model_flops,
    total_steps=max_steps,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

C:\Users\LENOVO\.conda\envs\testing_finetune_llm\lib\site-packages\accelerate\accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [16]:
training_output = trainer.train()

Inputs:  {'input_ids': tensor([], device='cuda:0', size=(1, 0)), 'attention_mask': tensor([], device='cuda:0', size=(1, 0)), 'labels': tensor([], device='cuda:0', size=(1, 0))}
Inputs - input_ids tensor([], device='cuda:0', size=(1, 0))
numel 0


Step,Training Loss,Validation Loss,Time,
120,2.600107,2.462791,-25.520080,10066313976440.591797
240,2.389148,2.282554,-52.177435,9973147767949.662109
360,2.138661,2.218792,-78.570429,9976442985052.505859
480,1.969670,2.166470,-104.926129,9981627576097.062500
600,1.886782,2.103992,-131.080618,10000057256322.964844
720,1.710067,2.111018,-157.078937,10022310150428.849609
840,1.651831,2.084747,-182.859735,10050183863467.626953
960,1.593966,2.087108,-208.832759,10061898847676.244141
1080,1.476995,2.077676,-234.760329,10072972061861.523438
1200,1.405734,2.071707,-260.744949,10079636737016.169922


Inputs:  {'input_ids': tensor([], device='cuda:0', size=(1, 0)), 'attention_mask': tensor([], device='cuda:0', size=(1, 0)), 'labels': tensor([], device='cuda:0', size=(1, 0))}
Inputs - input_ids tensor([], device='cuda:0', size=(1, 0))
numel 0
Inputs:  {'input_ids': tensor([], device='cuda:0', size=(1, 0)), 'attention_mask': tensor([], device='cuda:0', size=(1, 0)), 'labels': tensor([], device='cuda:0', size=(1, 0))}
Inputs - input_ids tensor([], device='cuda:0', size=(1, 0))
numel 0
Inputs:  {'input_ids': tensor([], device='cuda:0', size=(1, 0)), 'attention_mask': tensor([], device='cuda:0', size=(1, 0)), 'labels': tensor([], device='cuda:0', size=(1, 0))}
Inputs - input_ids tensor([], device='cuda:0', size=(1, 0))
numel 0
Inputs:  {'input_ids': tensor([], device='cuda:0', size=(1, 0)), 'attention_mask': tensor([], device='cuda:0', size=(1, 0)), 'labels': tensor([], device='cuda:0', size=(1, 0))}
Inputs - input_ids tensor([], device='cuda:0', size=(1, 0))
numel 0
Inputs:  {'input_ids

In [17]:
save_dir = f'{output_dir}/final'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

Saved model to: lamini_docs_3_steps/final3


In [18]:
finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)


In [19]:
finetuned_slightly_model.to(device) 


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [20]:
test_question = test_dataset[0]['question']
print("Question input (test):", test_question)

print("Finetuned slightly model's answer: ")
print(inference(test_question, finetuned_slightly_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): Is it possible to fine-tune Lamini on a specific dataset for text generation in legal documents?
Finetuned slightly model's answer: 
Yes, it is possible to fine-tune Lamini on a specific dataset for text generation in legal documents. Lamini is a language model that can be trained on a corpus of text generated by a large corpus of legal documents. Lamini can be fine-tuned on specific datasets or domains to ensure that the data is representative of the desired content. Additionally, L


In [21]:
test_answer = test_dataset[0]['answer']
print("Target answer output (test):", test_answer)

Target answer output (test): Lamini’s LLM Engine can help you fine-tune any model on huggingface or any OpenAI model.
